# Title: msticpy - GeoIP Lookup

## Introduction
This module contains two classes that allow you to look up the Geolocation of IP Addresses.

### MaxMind GeoIPLite
This product includes GeoLite2 data created by MaxMind, available from
<a href="https://www.maxmind.com">https://www.maxmind.com</a>.

This uses a local database which gives very fast lookups but you need to download updates regularly. Maxmind offers a free tier of this database, updated monthly. For greater accuracy and more detailed information they have varying levels of paid service. Please check out their site for more details.

To install Maxmind GeoIP library and database
```
!pip install maxminddb_geolite2```
or
```
!pip install maxminddb_geolite2 --user```

To uppdate the package and database:
```
!pip install maxminddb_geolite2 --upgrade [--user]```

### IPStack
This library uses services provided by ipstack.
<a href="https://ipstack.com">https://ipstack.com</a>

IPStack is an online service and also offers a free tier of their service. Again, the paid tiers offer greater accuracy, more detailed information and higher throughput. Please check out their site for more details.



<a id='contents'></a>
## Table of Contents
- [Maxmind GeoIP Lookup](#geoip_lookups)
- [IPStack GeoIP Lookup](#ipstack_lookups)
- [Dataframe input](#dataframe_input)
- [Creating your own GeoIP Class](#custom_lookup)
- [Calculating Geographical Distances](#calc_distance)

In [3]:
# Imports
import sys
MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)


from IPython.display import display
import pandas as pd

import msticpy.sectools as sectools
import msticpy.nbtools as mas
from msticpy.nbtools.entityschema import IpAddress, GeoLocation
from msticpy.sectools.geoip import GeoLiteLookup, IPStackLookup

<a id='geoip_lookups'></a>[Contents](#contents)
## Maxmind GeoIP Lite Loopup Class
Signature:
```
iplocation.lookup_ip(ip_address: str = None, 
                     ip_addr_list: collections.abc.Iterable = None,
                     ip_entity: msticpy.nbtools.entityschema.IpAddress = None)
Docstring:
Lookup IP location from IPStack web service.

Keyword Arguments:
    ip_address {str} -- a single address to look up (default: {None})
    ip_addr_list {Iterable} -- a collection of addresses to lookup (default: {None})
    ip_entity {IpAddress} -- an IpAddress entity

Returns:
    tuple(list{dict}, list{entity}) -- returns raw geolocation results and
        same results as IP/Geolocation entities
```

In [12]:
iplocation = GeoLiteLookup()
loc_result, ip_entity = iplocation.lookup_ip(ip_address='90.156.201.97')
print('Raw result')
display(loc_result)

print('IP Address Entity')
display(ip_entity[0])

Raw result


[{'continent': {'code': 'EU',
   'geoname_id': 6255148,
   'names': {'de': 'Europa',
    'en': 'Europe',
    'es': 'Europa',
    'fr': 'Europe',
    'ja': 'ヨーロッパ',
    'pt-BR': 'Europa',
    'ru': 'Европа',
    'zh-CN': '欧洲'}},
  'country': {'geoname_id': 2017370,
   'iso_code': 'RU',
   'names': {'de': 'Russland',
    'en': 'Russia',
    'es': 'Rusia',
    'fr': 'Russie',
    'ja': 'ロシア',
    'pt-BR': 'Rússia',
    'ru': 'Россия',
    'zh-CN': '俄罗斯'}},
  'location': {'accuracy_radius': 1000,
   'latitude': 55.7386,
   'longitude': 37.6068},
  'registered_country': {'geoname_id': 2017370,
   'iso_code': 'RU',
   'names': {'de': 'Russland',
    'en': 'Russia',
    'es': 'Rusia',
    'fr': 'Russie',
    'ja': 'ロシア',
    'pt-BR': 'Rússia',
    'ru': 'Россия',
    'zh-CN': '俄罗斯'}}}]

IP Address Entity


{"Address": "90.156.201.97", "Location": {"CountryCode": "RU", "CountryName": "Russia", "Longitude": 37.6068, "Latitude": 55.7386, "Type": "geolocation"}, "Type": "ipaddress"}

In [7]:
import socket
socket_info = socket.getaddrinfo("pypi.org",0,0,0,0)

ips = [res[4][0] for res in socket_info]
print(ips)

_, ip_entities = iplocation.lookup_ip(ip_addr_list=ips)
display(ip_entities)

['2a04:4e42:400::223', '2a04:4e42:200::223', '2a04:4e42:600::223', '2a04:4e42::223', '151.101.64.223', '151.101.0.223', '151.101.192.223', '151.101.128.223']


[{"Address": "2a04:4e42:400::223", "Location": {"Longitude": 8, "Latitude": 47, "Type": "geolocation"}, "Type": "ipaddress"},
 {"Address": "2a04:4e42:200::223", "Location": {"Longitude": 8, "Latitude": 47, "Type": "geolocation"}, "Type": "ipaddress"},
 {"Address": "2a04:4e42:600::223", "Location": {"Longitude": 8, "Latitude": 47, "Type": "geolocation"}, "Type": "ipaddress"},
 {"Address": "2a04:4e42::223", "Location": {"Longitude": 8, "Latitude": 47, "Type": "geolocation"}, "Type": "ipaddress"},
 {"Address": "151.101.64.223", "Location": {"CountryCode": "US", "CountryName": "United States", "Longitude": -97.822, "Latitude": 37.751, "Type": "geolocation"}, "Type": "ipaddress"},
 {"Address": "151.101.0.223", "Location": {"CountryCode": "US", "CountryName": "United States", "Longitude": -97.822, "Latitude": 37.751, "Type": "geolocation"}, "Type": "ipaddress"},
 {"Address": "151.101.192.223", "Location": {"CountryCode": "US", "CountryName": "United States", "Longitude": -97.822, "Latitude":

<a id='ipstack_lookups'></a>[Contents](#contents)
## IPStack Geo-lookup Class

#### Class Initialization

Note - requires IPStack API Key, Optional parameter bulk_lookup allows multiple IPs in a single request. This is only available with the paid Professional tier and above.
```
Init signature: IPStackLookup(api_key: str, bulk_lookup: bool = False)
Docstring:     
GeoIP Lookup using IPStack web service.

Raises:
    ConnectionError -- Invalid status returned from http request
    PermissionError -- Service refused request (e.g. requesting batch of addresses
            on free tier API key)
Init docstring:
Create a new instance of IPStackLookup.

Arguments:
    api_key {str} -- API Key from IPStack - see https://ipstack.com
    bulk_lookup {bool} -- For Professional and above tiers allowing you to
        submit multiple IPs in a single request.
        
```

#### lookup_ip method
```
Signature:
iplocation.lookup_ip(
    ['ip_address: str = None', 'ip_addr_list: collections.abc.Iterable = None', 'ip_entity: msticpy.nbtools.entityschema.IpAddress = None'],
) -> tuple
Docstring:
Lookup IP location from IPStack web service.

Keyword Arguments:
    ip_address {str} -- a single address to look up (default: {None})
    ip_addr_list {Iterable} -- a collection of addresses to lookup (default: {None})
    ip_entity {IpAddress} -- an IpAddress entity

Raises:
    ConnectionError -- Invalid status returned from http request
    PermissionError -- Service refused request (e.g. requesting batch of addresses
        on free tier API key)

Returns:
    tuple(list{dict}, list{entity}) -- returns raw geolocation results and
        same results as IP/Geolocation entities
```

<a id='IPStack'></a>[Contents](#contents)
### You will need a IPStack API key
You will get more detailed results and a higher throughput allowance if you have a paid tier. See IPStack website for more details

In [4]:
# Enter your IPStack Key here
ips_key = mas.GetEnvironmentKey(env_var='IPSTACK_API_KEY',
                           help_str='To obtain an API key sign up here https://www.ipstack.com/',
                           prompt='IPStack API key:')
ips_key.display()

HTML(value='To obtain an API key sign up here https://www.ipstack.com/')

In [11]:
iplocation = IPStackLookup(api_key=ips_key.value)
loc_result, ip_entity = iplocation.lookup_ip(ip_address='90.156.201.97')
print('Raw result')
display(loc_result)

print('IP Address Entity')
display(ip_entity[0])

Raw result


[({'ip': '90.156.201.97',
   'type': 'ipv4',
   'continent_code': 'EU',
   'continent_name': 'Europe',
   'country_code': 'RU',
   'country_name': 'Russia',
   'region_code': None,
   'region_name': None,
   'city': None,
   'zip': None,
   'latitude': 55.7386,
   'longitude': 37.6068,
   'location': {'geoname_id': None,
    'capital': 'Moscow',
    'languages': [{'code': 'ru', 'name': 'Russian', 'native': 'Русский'}],
    'country_flag': 'http://assets.ipstack.com/flags/ru.svg',
    'country_flag_emoji': '🇷🇺',
    'country_flag_emoji_unicode': 'U+1F1F7 U+1F1FA',
    'calling_code': '7',
    'is_eu': False}},
  200)]

IP Address Entity


{"Address": "90.156.201.97", "Location": {"CountryCode": "RU", "CountryName": "Russia", "Longitude": 37.6068, "Latitude": 55.7386, "Type": "geolocation"}, "Type": "ipaddress"}

In [13]:
loc_result, ip_entities = iplocation.lookup_ip(ip_addr_list=ips)
print('Raw results')
display(loc_result)

print('IP Address Entities')
display(ip_entities)

Raw results


[({'ip': '2a04:4e42:400::223',
   'type': 'ipv6',
   'continent_code': 'EU',
   'continent_name': 'Europe',
   'country_code': None,
   'country_name': None,
   'region_code': None,
   'region_name': None,
   'city': None,
   'zip': None,
   'latitude': 47,
   'longitude': 8,
   'location': {'geoname_id': None,
    'capital': None,
    'languages': None,
    'country_flag': None,
    'country_flag_emoji': None,
    'country_flag_emoji_unicode': None,
    'calling_code': None,
    'is_eu': False}},
  200),
 ({'ip': '2a04:4e42:200::223',
   'type': 'ipv6',
   'continent_code': 'EU',
   'continent_name': 'Europe',
   'country_code': None,
   'country_name': None,
   'region_code': None,
   'region_name': None,
   'city': None,
   'zip': None,
   'latitude': 47,
   'longitude': 8,
   'location': {'geoname_id': None,
    'capital': None,
    'languages': None,
    'country_flag': None,
    'country_flag_emoji': None,
    'country_flag_emoji_unicode': None,
    'calling_code': None,
    'is_

IP Address Entities


[{"Address": "2a04:4e42:400::223", "Location": {"Longitude": 8, "Latitude": 47, "Type": "geolocation"}, "Type": "ipaddress"},
 {"Address": "2a04:4e42:200::223", "Location": {"Longitude": 8, "Latitude": 47, "Type": "geolocation"}, "Type": "ipaddress"},
 {"Address": "2a04:4e42:600::223", "Location": {"Longitude": 8, "Latitude": 47, "Type": "geolocation"}, "Type": "ipaddress"},
 {"Address": "2a04:4e42::223", "Location": {"Longitude": 8, "Latitude": 47, "Type": "geolocation"}, "Type": "ipaddress"},
 {"Address": "151.101.64.223", "Location": {"CountryCode": "US", "CountryName": "United States", "Longitude": -97.822, "Latitude": 37.751, "Type": "geolocation"}, "Type": "ipaddress"},
 {"Address": "151.101.0.223", "Location": {"CountryCode": "US", "CountryName": "United States", "Longitude": -97.822, "Latitude": 37.751, "Type": "geolocation"}, "Type": "ipaddress"},
 {"Address": "151.101.192.223", "Location": {"CountryCode": "US", "CountryName": "United States", "Longitude": -97.822, "Latitude":

<a id='dataframe_input'></a>[Contents](#contents)
## Taking input from a pandas DataFrame

The base class for both implementations has a method that sources the ip addresses from a dataframe column and returns a new dataframe with the location information merged with the input frame
```
Signature: iplocation.df_lookup_ip(data: pandas.core.frame.DataFrame, column: str)
Docstring:
Lookup Geolocation data from a pandas Dataframe.

Keyword Arguments:
    data {pd.DataFrame} -- pandas dataframe containing IpAddress column
    column {str} -- the name of the dataframe column to use as a source
```

<a id='custom_lookup'></a>[Contents](#contents)
## Creating a Custom GeopIP Lookup Class

You can derive a class that implements the same operations to use with a different GeoIP service.

The class signature is as follows:
```
class GeoIpLookup(ABC):
    """Abstract base class for GeoIP Lookup classes."""

    @abstractmethod
    def lookup_ip(self, ip_address: str = None, ip_addr_list: Iterable = None,
                  ip_entity: IpAddress = None):
        """
        Lookup IP location.

        Keyword Arguments:
            ip_address {str} -- a single address to look up (default: {None})
            ip_addr_list {Iterable} -- a collection of addresses to lookup (default: {None})
            ip_entity {IpAddress} -- an IpAddress entity

        Returns:
            tuple(list{dict}, list{entity}) -- returns raw geolocation results and
                same results as IP/Geolocation entities

        """
```
You should override the lookup_ip method implementing your own method of geoip lookup.

<a id='calc_distance'></a>[Contents](#contents)
## Calculating Geographical Distances

Use the geo_distance function from msticpy.sectools.geoip to calculated distances between two locations.
I am indebted to Martin Thoma who posted this solution (which I've modified slightly) on Stackoverflow.


```
Signature: geo_distance(origin: Tuple[float, float], destination: Tuple[float, float]) -> float
Docstring:
Calculate the Haversine distance.

Author: Martin Thoma - stackoverflow

Parameters
----------
origin : tuple of float
    (lat, long)
destination : tuple of float
    (lat, long)

Returns
-------
distance_in_km : float
```


Or where you have source and destination IpAddress entities, you can use the wrapper entity_distance.
```
Signature:
entity_distance(
    ['ip_src: msticpy.nbtools.entityschema.IpAddress', 'ip_dest: msticpy.nbtools.entityschema.IpAddress'],
) -> float
Docstring:
Return distance between two IP Entities.

Arguments:
    ip_src {IpAddress} -- Source IpAddress Entity
    ip_dest {IpAddress} -- Destination IpAddress Entity

Raises:
    AttributeError -- if either entity has no location information

Returns:
    float -- Distance in kilometers.
```

In [24]:
from msticpy.sectools.geoip import geo_distance
_, ip_entity1 = iplocation.lookup_ip(ip_address='90.156.201.97')
_, ip_entity2 = iplocation.lookup_ip(ip_address='151.101.64.223')

print(ip_entity1[0])
print(ip_entity2[0])
dist = geo_distance(origin=(ip_entity1[0].Location.Latitude, ip_entity1[0].Location.Longitude),
                    destination=(ip_entity2[0].Location.Latitude, ip_entity2[0].Location.Longitude))
print(f'\nDistance between IP Locations = {round(dist, 1)}km')

{ 'Address': '90.156.201.97',
  'Location': { 'CountryCode': 'RU',
                'CountryName': 'Russia',
                'Latitude': 55.7386,
                'Longitude': 37.6068,
                'Type': 'geolocation'},
  'Type': 'ipaddress'}
{ 'Address': '151.101.64.223',
  'Location': { 'CountryCode': 'US',
                'CountryName': 'United States',
                'Latitude': 37.751,
                'Longitude': -97.822,
                'Type': 'geolocation'},
  'Type': 'ipaddress'}

Distance between IP Locations = 8796.8km


In [31]:
dist2 = entity_distance(ip_entity1[0],ip_entity2[0])
print(f'\nDistance between IP Entity Locations = {round(dist2, 1)}km')


Distance between IP Locations = 8796.8km
